# Initialisierung

In [ ]:
# Packages installieren (ggf. beim ersten Lauf aktivieren, danach auskommentieren)
#install.packages("tidyverse")
#install.packages("leaflet")
#install.packages("corrplot")

# Packages laden
library(readr)
library(dplyr)
library(tidyr)
library(lubridate)
library(leaflet)
library(corrplot)
library(ggplot2)

# Konfiguration
base_url = "https://raw.githubusercontent.com/miracum/c19ss2021/master/uebung-5/"

# Data Load & Prep

## Bielefelder Daten laden & preppen

In [ ]:
# Daten laden
patients.bielefeld  <- read_csv(paste(base_url, "rohdaten/bielefeld/csv/patients.csv",   sep=""), col_types = cols(BIRTHDATE = col_date(format = "%Y-%m-%d")))
diagnoses.bielefeld <- read_csv(paste(base_url, "rohdaten/bielefeld/csv/conditions.csv", sep=""), col_types = cols(START = col_date(format = "%Y-%m-%d"), STOP = col_date(format = "%Y-%m-%d")))

# Alter ergänzen
patients.bielefeld <- mutate(patients.bielefeld, AGE = case_when(is.na(DEATHDATE) ~ as.period(interval(start = BIRTHDATE, end = Sys.Date()))$year, TRUE ~ as.period(interval(start = BIRTHDATE, end = DEATHDATE))$year))

# Patienten mit COVID-19 selektieren
cohort.patients.bielefeld <- diagnoses.bielefeld %>% 
    filter(DESCRIPTION == "COVID-19") %>% 
    select(PATIENT) %>% distinct()

# Demographische Angaben ergänzen
cohort.patients.bielefeld <- cohort.patients.bielefeld %>% 
    inner_join(patients.bielefeld, by=c("PATIENT" = "Id")) %>%
    select(PATIENT, BIRTHDATE, DEATHDATE, GENDER, AGE, CITY)

# Vitalstatus ergänzen
cohort.patients.bielefeld <- mutate(cohort.patients.bielefeld, VITALSTATUS = case_when(is.na(DEATHDATE) ~ "ALIVE", TRUE ~ "DEAD"))

# Begleitdiagnosen bestimmen
cohort.secondary_diagnoses.bielefeld <- cohort.patients.bielefeld %>% 
            inner_join(diagnoses.bielefeld, by="PATIENT") %>%
            filter(!(DESCRIPTION %in% c("COVID-19", "Suspected COVID-19")))

# Zu untersuchende Nebendiagnosen (Adipositas, Erschöpfung und Atemnot) auslesen
tmp <- cohort.secondary_diagnoses.bielefeld %>% 
       filter(DESCRIPTION %in% c("Body mass index 30+ - obesity (finding)","Fatigue (finding)","Respiratory distress (finding)")) %>%
       select(PATIENT, DESCRIPTION) %>%
       mutate(DIAGNOSISPRESENT = 1)

# Diagnosen zu Spalten pivotieren
tmp <- pivot_wider(tmp, names_from=DESCRIPTION, values_from=DIAGNOSISPRESENT, values_fill=NA)

# Spalten umbenennen
colnames(tmp) <- c("PATIENT", "OBESITY", "FATIGUE", "RESPIRATORY_DISTRESS")

# Stammdaten & zu untersuchende Diagnosen zusammenführen
cohort.dataset.bielefeld <- cohort.patients.bielefeld %>% left_join(tmp, by = "PATIENT")
head(cohort.dataset.bielefeld)

## Mannheimer Daten laden & preppen

### Ihre Aufgabe: übernehmen Sie den Bielefelder Code & passen Sie ihn auf Mannheim an!

## Leipziger Daten laden & preppen

### Ihre Aufgabe: übernehmen Sie den Bielefelder Code & passen Sie ihn auf Leipzig an!

# Generierung lokal aggregierter Zwischenergebnisse

## Zwischenergebnisse für Bielefeld

In [ ]:
# Lokal aggregierte Zwischenergebnisse für Bielefeld generieren
cohort.localresult.bielefeld <- cohort.dataset.bielefeld %>%
    group_by(VITALSTATUS) %>%
    summarize(OBESITY = sum(OBESITY, na.rm=TRUE), 
              FATIGUE = sum(FATIGUE, na.rm=TRUE), 
              RESPIRATORY_DISTRESS = sum(RESPIRATORY_DISTRESS, na.rm=TRUE))
head(cohort.localresult.bielefeld)

### Ihre Aufgabe: übernehmen Sie den Bielefelder Code und passen Sie ihn für Mannheim & Leipzig an!

# Zusammenführung der lokalen Zwischenergebnisse & Auswertung

## Ihre Aufgabe: führen Sie die 3 Zwischenergebnisse zusammen!

## Ihre Aufgabe: führen Sie den Chi-Quadrat-Test für die zusammengeführten Zwischenergebnisse durch!